In [151]:
import pandas as pd
import numpy as np 
!pip install unit-converter

# Clean the data source

In [381]:
data = pd.read_csv("DRI - Complete.csv",engine='python')
fp = pd.read_csv("foodprice.csv") 
data_up = pd.read_csv("DRI - upper.csv", engine='python') 
nutrifact = pd.read_csv("nutrifact.csv") 
# Preview the first 5 lines of the loaded data 
data.rename(columns = {'Life Stage\nGroup':'age'}, inplace = True) 
data.rename(columns = {'Pregnanc':'Pregnancy'}, inplace = True) 
data.head()
# fp.drop(['unit'])
fp.Quantity = fp.Quantity.astype(str)
fp['unit']=fp['Quantity']+fp['Units']
fp.Quantity = fp.Quantity.astype(float)

#convert all the units
fp.loc[fp.Units == 'oz','Quantity']=fp.loc[fp.Units == 'oz','Quantity']*28
fp.loc[fp.Units == 'kg','Quantity']=fp.loc[fp.Units == 'kg','Quantity']*1000
fp.loc[fp.Units == 'lbs','Quantity']=fp.loc[fp.Units == 'lbs','Quantity']*453.592
fp.loc[fp.Units == 'mL','Quantity']=fp.loc[fp.Units == 'mL','Quantity']*1
fp.loc[fp.Units == 'lb','Quantity']=fp.loc[fp.Units == 'lb','Quantity']*453.592
fp.loc[fp.Units == 'ct','Quantity']=fp.loc[fp.Units == 'ct','Quantity']*200
fp.loc[fp.Units == 'fl oz','Quantity']=fp.loc[fp.Units == 'fl oz','Quantity']*29.5735
fp['Units']="g"
fp.Quantity = fp.Quantity.astype(str)
fp['unit']=fp['Quantity']+fp['Units']
fp.Quantity = fp.Quantity.astype(float)
fp['portion']=fp['Quantity']/100
result = pd.concat([fp,nutrifact], axis=1, join='inner')
# result = pd.concat([fp,nutrifact], axis=1, sort=False)
result.fillna(0, inplace=True)
result['price_per_portion']=result['Price-US']/result['portion']
result=result.drop(['Portion','Price-US','unit','portion','Units',"Quantity"], axis=1)
result = result.loc[:,~result.columns.duplicated()]
result=result.set_index('Food')
result
result

,Nutrients,Calcium (mg/d),"Carbohydrate,by difference g","Carotene,alpha ug","Carotene,beta ug",Choline (g/d),Choline (g/d).1,"Copper (_g/d),Cu mg",Energy kcal,"Fatty Acids,total saturated g",...,Iron (mg/d),"Megnesium,Mg","Potassium,K mg",Protein g,Sodium (g/d),"Sugars,total including NLEA g",Total lipid(fat) g,Vitamin A (_g/d)a,Vitamin C (mg/d),price_per_portion
Food,,,,,,,,,,,,,,,,,,,,,
tofu,496449.0,0.0,20.00,0.0,0.0,0.0,0.0,0.000,175.0,1.25,...,0.00,0.0,0.0,2.50,1425.0,25.00,7.50,0.0,0.0,0.399554
lentils,341206.0,34.0,23.47,0.0,10.0,0.0,16.7,0.160,156.0,0.00,...,1.92,29.0,232.0,5.00,196.0,0.59,0.00,0.0,0.6,0.310268
beans,704294.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000,0.0,0.00,...,0.00,0.0,0.0,0.00,39333.0,0.00,0.00,0.0,0.0,0.220982
carrots,450701.0,26.0,8.97,0.0,0.0,0.0,0.0,0.000,38.0,0.00,...,0.46,0.0,0.0,1.28,77.0,6.41,0.00,7051.0,7.7,0.099330
apples,592260.0,7.0,12.86,0.0,0.0,0.0,0.0,0.000,50.0,0.00,...,0.13,0.0,74.0,0.00,60.0,10.00,0.00,36.0,4.7,0.440000
oranges,578523.0,45.0,12.34,0.0,0.0,0.0,0.0,0.000,49.0,1.90,...,0.12,0.0,0.0,0.65,1.0,8.44,0.15,260.0,59.2,0.440000
beets,384502.0,0.0,6.67,0.0,0.0,0.0,0.0,0.000,33.0,0.00,...,0.00,0.0,0.0,0.83,208.0,5.00,0.00,0.0,0.0,1.067857
celery,447185.0,36.0,3.64,0.0,0.0,0.0,0.0,0.000,14.0,0.00,...,0.33,0.0,0.0,0.00,105.0,1.82,0.00,182.0,8.2,0.595000
onions,342625.0,23.0,9.34,0.0,1.0,0.0,6.1,0.039,40.0,0.00,...,0.21,10.0,146.0,1.10,4.0,4.24,0.10,0.0,0.0,0.320000


# get the price of food

In [210]:

# result['Food'],result['price_per_portion']


price=result['price_per_portion']
price

Food
tofu                 0.399554
lentils              0.310268
beans                0.220982
carrots              0.099330
apples               0.440000
oranges              0.440000
beets                1.067857
celery               0.595000
onions               0.320000
brocolli             0.534598
lemon                0.339512
veggie broth         0.143973
rice                 0.620000
wheat flour          0.087965
whole wheat flour    0.162701
vegetable oil        0.170387
olive oil            0.534999
margarine            1.100108
sugar                0.377232
potatoes             0.320000
pasta                0.147959
Soy Milk             0.241561
Loaf of Bread        0.784000
Almond milk          0.167411
Rice milk            0.322545
brown rice           0.251180
wheat bread          0.296131
Name: price_per_portion, dtype: float64

In [211]:
D=result.transpose()


# get the constrains, minimum and maximum

In [348]:
age_range = data['Life Stage Group'].str.split(r'-')
age_range
data['age_range']=age_range
data_up['age_range']=age_range
data_up=data_up.replace('ND',0)
data=data.replace('ND',0)
data
def get_dri_min(age,condition):
    data_copy = data[data[condition]==1]
    data_copy=data_copy.fillna(0)
    data_copy=data_copy.reset_index()
    for x in data_copy.index:
        if int(data_copy['age_range'][x][1])>age:
            nutri_min= data_copy.loc[x]
            data_copy=data_copy.drop(['age_range','Life Stage Group','Infants','Children','Males','Females','Pregnanc','Lactation'],axis=1)
            return data_copy.loc[x]
#conditions:Infant, Children, Males, Females, Pregancy,Lactation
get_dri_Low(32,'Lactation')
Constrain_min=get_dri_Low(32,'Lactation')
def get_dri_max(age,condition):
    data_copy = data_up[data[condition]==1]
    data_copy=data_copy.dropna()
    data_copy=data_copy.reset_index()
    for x in data_copy.index:
        if int(data_copy['age_range'][x][1])>age:
            data_copy=data_copy.drop(['age_range','Life Stage Group','Infants','Children','Males','Females','Pregnanc','Lactation'],axis=1)
            return data_copy.loc[x]
#conditions:Infant, Children, Males, Females, Pregancy,Lactation
get_dri_max(22,'Lactation')

Constrain_max= get_dri_max(22,'Lactation')
Constrain_max= Constrain_max.astype(np.float16)

Constrain_max,Constrain_min

(index                     20.000000
 Vitamin A (_g/d)a       3000.000000
 Vitamin C (mg/d)        2000.000000
 Vitamin D (_g/d)         100.000000
 Vitamin E (mg/d)b,c     1000.000000
 Vitamin K                  0.000000
 Thiamin                    0.000000
 Riboflavin                 0.000000
 Niacin (mg/d)c            35.000000
 Vitamin B6 (mg/d)        100.000000
 Folate (_g/d)c          1000.000000
 Vitamin B12                0.000000
 Pantothenic Acid           0.000000
 Biotin                     0.000000
 Choline (g/d)              3.500000
 Carotenoidsd               0.000000
 Arsenica                   0.000000
 Boron (mg/d)              20.000000
 Calcium (mg/d)          2500.000000
 Chromium                   0.000000
 Copper (_g/d)          10000.000000
 Fluoride (mg/d)           10.000000
 Iodine (_g/d)           1100.000000
 Iron (mg/d)               45.000000
 Magnesium (mg/d)b        350.000000
 Manganese (mg/d)          11.000000
 lbdenum (_g/d)          2000.000000
 

# Constraint matrix (A)

In [363]:
Aall = D[c.index].fillna(0)
Aall


for items in Constrain_min.index.tolist():
    if items not in Aall.index.tolist():
        print(items)
        print(np.shape(Constrain_min))
        Constrain_min=Constrain_min.drop(index=items)

for items in Constrain_max.index.tolist():
    if items not in Aall.index.tolist():
        print(items)
        print(np.shape(Constrain_min))
        Constrain_max=Constrain_max.drop(index=items)


b.fillna(0, inplace=True)

b = pd.concat([Constrain_min,-Constrain_max,])

# Drop rows of A that we don't have constraints for.
# Amin = Aall.loc[Constrain_min.index.tolist()]
Amin=Aall
for items in Amin.index.tolist():
    if items not in Constrain_min.index.tolist():
        Amin=Amin.drop(index=items)
Amax=Aall
for items in Amax.index.tolist():
    if items not in Constrain_max.index.tolist():
        Amax=Amax.drop(index=items,axis=1)
Amax


# for items in b.index.tolist():
#     print(items)
#     if items not in A.index.tolist():
#         b=b.drop(items,axis=0)
#         print (np.shape(b))
        

# for items in b.index.tolist():
#     if items not in A.index.tolist():
#         b=b.drop(index=items)
        

# Amax = Aall.loc[Constrain_max.index]

# Minimum requirements involve multiplying constraint by -1 to make <=.



# for items in b.index.tolist():
#     if items not in A.index.tolist():
#         B=b.drop(index=items,axis=1)

A = pd.concat([Amin,-Amax])

In [377]:
# !pip install --upgrade pip
# !pip install -r requirements.txt
# !pip install pint

# import sys
# sys.path.append('../MinCostDiet/')

# user = "Nan"
# apikey = {'Nan':"4SBJVP1hlkc3reMMAHKWgGRK43CdhFLChRdgoaHl"}
# import fooddatacentral as fdc

# group = "F 19-30"

# # Define *minimums*
# bmin = pd.read_csv('./diet_minimums.csv').set_index('Nutrition')[group]

# # Define *maximums*
# bmax = pd.read_csv('./diet_maximums.csv').set_index('Nutrition')[group]

# # Inspect:
# bmin


In [378]:
b.index.tolist()

['Calcium (mg/d)',
 'Iron (mg/d)',
 'Sodium (g/d)',
 'Vitamin A (_g/d)a',
 'Vitamin C (mg/d)',
 'Choline (g/d)',
 'Vitamin A (_g/d)a',
 'Vitamin C (mg/d)',
 'Choline (g/d)',
 'Calcium (mg/d)',
 'Iron (mg/d)',
 'Sodium (g/d)']

In [367]:
np.linalg.norm(price)

2.4262927269048813

# Objective Function (C)

In [392]:
from  scipy.optimize import linprog as lp
import numpy as np

c = price.apply(lambda x:np.linalg.norm(x)).dropna()


# Compile list that we have both prices and nutritional info for; drop if either missing
use = list(set(c.index.tolist()).intersection(A.columns.tolist()))
c = c[use]
# from  scipy.optimize import linprog as lp
# import numpy as np
c
tol = 1e-6 # Numbers in solution smaller than this (in absolute value) treated as zeros



result = lp(c, -A, -b, method='interior-point')
result


     con: array([], dtype=float64)
     fun: 59.209781926207
 message: 'The algorithm terminated successfully and determined that the problem is infeasible.'
     nit: 6
   slack: array([ 2.03392052e+03,  1.46546613e+02,  2.13246865e+01,  4.64011627e+04,
        9.36591896e+03, -3.14038552e+02, -3.39205201e+01,  1.84445339e+03,
       -1.93246865e+01, -4.52011627e+04, -9.44091896e+03, -2.33660667e+02])
  status: 2
 success: False
       x: array([  0.62522328,   0.        ,   3.13569845,   0.97856932,
         0.60517118,   0.73453269,   1.09288557,   2.82702806,
         1.48860392,   0.88561036,  14.83318771,   0.        ,
         0.99774421,   0.8137461 ,   0.63487658,   0.637346  ,
         0.38490132,   1.4430676 ,   1.03667132,   0.91618677,
         0.        , 120.32049432,   0.7660979 ,   0.53225871,
         8.11285846,   9.3852166 ,  22.3164478 ])

In [394]:
print("Cost of diet for %s is $%4.2f per day." % (group,result.fun))

diet = pd.Series(result.x,index=c.index)

print("\nYou'll be eating (in 100s of grams or milliliters):")

Cost of diet for F 19-30 is $59.21 per day.

You'll be eating (in 100s of grams or milliliters):


In [11]:
# useless functions



(Life Stage Group         19�30�y
 Vitamin A ( g/d)a          3,000
 Vitamin C (mg/d)           2,000
 Vitamin D ( g/d)             100
 Vitamin E (mg/d)b,c        1,000
 Vitamin K                     ND
 Thiamin                       ND
 Riboflavin                    ND
 Niacin (mg/d)c                35
 Vitamin B6 (mg/d)            100
 Folate ( g/d)c             1,000
 Vitamin B12                   ND
 Pantothenic Acid              ND
 Biotin                        ND
 Choline (g/d)                3.5
 Carotenoidsd                  ND
 Arsenica                      ND
 Boron (mg/d)                  20
 Calcium (mg/d)             2,500
 Chromium                      ND
 Copper ( g/d)             10,000
 Fluoride (mg/d)               10
 Iodine ( g/d)              1,100
 Iron(mg/d)                    45
 Magnesium (mg/d)b            350
 Manganese (mg/d)              11
 Molybdenum ( g/d)          2,000
 Nickel (mg/d)                  1
 Phosphorus (g/d)               4
 Selenium ( g/